In [1]:
import pandas as pd
df = pd.read_csv('df_final_cpa.csv')
df

,DATA_DE_INÍCIO_ATIVIDADE,CNAE_FISCAL_PRINCIPAL,CNAE_FISCAL_SECUNDÁRIA,MUNICÍPIO
0,2023,7319003,5819100,2457
1,2023,5611201,1032599,7097
2,2023,9001902,7729202,301
3,2023,5611203,5611204,7149
4,2023,9001902,5229099,391
...,...,...,...,...
51998540,1907,8513900,8599699,7107
51998541,1903,9430800,9499500,921
51998542,1903,9430800,9493600,921
51998543,1901,6912500,9609299,6645


In [2]:
# ANO1
# CRIANDO A COLUNA 'ANO1' COM O VALOR MAIS ANTIGO (MIN) DE ANO DE ABERTURA DE CNAE_FISCAL_PRINCIPAL POR MUNICÍPIO:
df['ANO1'] = df.groupby('MUNICÍPIO')['DATA_DE_INÍCIO_ATIVIDADE'].transform('min')
df = df.sort_values('ANO1')
df

,DATA_DE_INÍCIO_ATIVIDADE,CNAE_FISCAL_PRINCIPAL,CNAE_FISCAL_SECUNDÁRIA,MUNICÍPIO,ANO1
51998544,1891,1321900,7490104,4675,1891
21880183,2020,4712100,4789005,4675,1891
50408370,1994,4930202,4511102,4675,1891
50408371,1994,4930202,4930201,4675,1891
50408372,1994,4930202,5212500,4675,1891
...,...,...,...,...,...
6385051,2022,4729699,4721103,320,2002
31452899,2017,4530703,4923002,320,2002
31452898,2017,4530703,4782201,320,2002
11426571,2021,4712100,4721103,320,2002


In [3]:
# MATURIDADE_CP
# MATURIDADE_MUN
# Criando a variável maturidade para os CNAE_FISCAL_PRINCIPAL:

df['MATURIDADE_CP'] = df['DATA_DE_INÍCIO_ATIVIDADE'] - df['ANO1']

# Agrupar por município e encontrar o maior valor de maturidade em cada grupo para termos a MATURIDADE_MUN

maturidade_mun = df.groupby('MUNICÍPIO')['MATURIDADE_CP'].max()

# Converter a série em DataFrame
df_maior_maturidade = pd.DataFrame({'MUNICÍPIO': maturidade_mun.index,
                                    'MATURIDADE_MUN': maturidade_mun.values})

# Mesclar o DataFrame original com o DataFrame da menor maturidade
df = pd.merge(df, df_maior_maturidade, on='MUNICÍPIO', how='left')
df

,DATA_DE_INÍCIO_ATIVIDADE,CNAE_FISCAL_PRINCIPAL,CNAE_FISCAL_SECUNDÁRIA,MUNICÍPIO,ANO1,MATURIDADE_CP,MATURIDADE_MUN
0,1891,1321900,7490104,4675,1891,0,132
1,2020,4712100,4789005,4675,1891,129,132
2,1994,4930202,4511102,4675,1891,103,132
3,1994,4930202,4930201,4675,1891,103,132
4,1994,4930202,5212500,4675,1891,103,132
...,...,...,...,...,...,...,...
51998540,2022,4729699,4721103,320,2002,20,21
51998541,2017,4530703,4923002,320,2002,15,21
51998542,2017,4530703,4782201,320,2002,15,21
51998543,2021,4712100,4721103,320,2002,19,21


In [4]:
# Solicite ao usuário que forneça o valor do MUNICÍPIO
municipio_informado = input("Informe o código do MUNICÍPIO: ")

# Filtrar as linhas com o MUNICÍPIO '6001' e pegar a relação de CNAE_FISCAL_PRINCIPAL
cnaes_municipio_informado = df[df['MUNICÍPIO'] == int(municipio_informado)]['CNAE_FISCAL_PRINCIPAL']

# Verificar se existem registros para o código do MUNICÍPIO informado
if not cnaes_municipio_informado.empty:
    # Filtrar as linhas com a mesma MATURIDADE_MUN do código do MUNICÍPIO informado
    maturidade_municipio = df[df['MUNICÍPIO'] == int(municipio_informado)]['MATURIDADE_MUN']
    cnaes_mesma_maturidade = df[df['MATURIDADE_MUN'] == maturidade_municipio.iloc[0]]['CNAE_FISCAL_PRINCIPAL']

    # Subtrair 'cnaes_mesma_maturidade' de 'cnaes_municipio_informado' para encontrar os inexistentes
    cnaes_inexistentes = cnaes_mesma_maturidade[~cnaes_mesma_maturidade.isin(cnaes_municipio_informado)].unique()

    # Exibir os resultados
    # print("CNAE_FISCAL_PRINCIPAL do MUNICÍPIO '6001':")
    # print(cnaes_municipio_informado)

    # print("\nCNAE_FISCAL_PRINCIPAL com a mesma MATURIDADE_MUN:")
    # print(cnaes_mesma_maturidade)

    print("\nCNAE_FISCAL_PRINCIPAL inexistentes:")
    print(cnaes_inexistentes)
else:
    print(f"Nenhum registro encontrado para o MUNICÍPIO {municipio_informado}.")

Informe o código do MUNICÍPIO:  8569



CNAE_FISCAL_PRINCIPAL inexistentes:
[4649499 6920602 7119703 4221905 2319200  155505  162801 6463800 1412603
  132600 4645101 3314705 4541201 1069400 1411801 7729201 8533300 7210000
 8219901  155503  723501 3812200 7830200 2812700 2949299 6391700 8650005
 1413401 2733300 2219600 6550200 4923001  111302 4511103 2651500 4634602
 1121600 4647801 8424800 5914600 6450600 4671100 1623400 8520100 4530701
 2431800 4329104 7740300 4679604 5812302 2229301  152102 7820500 4635499
  159802 4929999 4530702 9529103 2840200 5911101 3702900  134200 2071100
  893200 8430200  119906 4729602 6201502 8640203 4669999 8621601 7912100
  119999 5231102  116499 4312600 6621502 3104700 4647802 4637199 5510802
  321304 8299701 4636201 4634603 2539002 1413403  724301 6630400 5912001
 1112700 3600601 3316301 7990200 3311200 2599301 4687701 4311801 4641902
  810002 4665600 8292000  133499 3250706 6499999 2852600 3250709 4635401
 4643501 2941700  139304 4922102 4674500 4662100 1340599 4221904  139399
  122900 46338

In [5]:
# Gerando um DataFrame:
df_camaqua = pd.DataFrame(cnaes_inexistentes)
df_camaqua

,0
0,4649499
1,6920602
2,7119703
3,4221905
4,2319200
...,...
646,4684202
647,1099606
648,2052500
649,133407


In [6]:
# Salvar a planilha em excel para conferência
df_camaqua = df_camaqua.to_csv('df_camaqua.csv', index=False)